In [9]:
import string
# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

def plot_data(df_list:list[pd.DataFrame], legend:list, y_data:list, x_data:string, h_dist:bool = True):
    
    
    for y in y_data:
        fig = go.Figure()    
        for df in df_list:
            fig = fig.add_trace(go.Scatter(x = df[x_data],y=df[y]))
        fig.show()
    
    # if h_dist:
    #     plt.figure(figsize=(100, 100))
    #     ax_n = len(y_data)
    #     for y in y_data:
    #         index = y_data.index(y)
    #         plt.subplot(1,ax_n,index+1)
    #         for df in df_list:
    #             plt.plot(df[x_data], df[y])
    #             plt.ylabel(y)
    #             plt.xlabel(x_data)
    #             plt.legend(legend)
    # else:
    #     plt.figure(figsize=(10, 20))
    #     ax_n = len(y_data)
    #     for y in y_data:
    #         index = y_data.index(y)
    #         plt.subplot(ax_n,1,index+1)
    #         for df in df_list:
    #             plt.plot(df[x_data], df[y])
    #             plt.ylabel(y)
    #             plt.xlabel(x_data)
    #             plt.legend(legend)
    
    
    # plt.show()

In [10]:
# Manipulando arquivo de ephemeris

from src.common.vector import Vector
import pandas as pd


def read_ephemeris(filepath):
    file = open(filepath, "r")
    lines = file.readlines()

    t = []
    rx = []
    ry = []
    rz = []
    vx = []
    vy = []
    vz = []

    for line in lines[11:-3]:
        splitted_line = line.split()
        t.append(float(splitted_line[0]))
        rx.append(float(splitted_line[1]))
        ry.append(float(splitted_line[2]))
        rz.append(float(splitted_line[3]))
        vx.append(float(splitted_line[4]))
        vy.append(float(splitted_line[5]))
        vz.append(float(splitted_line[6]))

    df = pd.DataFrame(
        data={
            "Time [s]": t,
            "Rx [km]": rx,
            "Ry [km]": ry,
            "Rz [km]": rz,
            "Vx [km/s]": vx,
            "Vy [km/s]": vy,
            "Vz [km/s]": vz,
        }
    )
    return df



In [11]:
from src.orbit.orbit import rv_from_r0v0

df_900 = read_ephemeris("Dados\\Ephemeris\\VCUB_TEST_900s.e")
df_60 = read_ephemeris("Dados\\Ephemeris\\VCUB_TEST_60s.e")

int_t = []
int_rx = []
int_ry = []
int_rz = []
int_vx = []
int_vy = []
int_vz = []
int_erro_rx = []
int_erro_ry = []
int_erro_rz = []
int_erro_vx = []
int_erro_vy = []
int_erro_vz = []

for index, row in df_60.iterrows():
    t = row['Time [s]']
    base_row = df_900[df_900['Time [s]'] <= t].tail(1)
    dt = t - base_row['Time [s]'].values[0]
    r0 = Vector(base_row['Rx [km]'].values[0],base_row['Ry [km]'].values[0],base_row['Rz [km]'].values[0])
    v0 = Vector(base_row['Vx [km/s]'].values[0],base_row['Vy [km/s]'].values[0],base_row['Vz [km/s]'].values[0])
    if index > 0:
        r,v = rv_from_r0v0(r0,v0,dt)
    else:
        r = Vector(row['Rx [km]'],row['Ry [km]'],row['Rz [km]'])
        v = Vector(row['Vx [km/s]'],row['Vy [km/s]'],row['Vz [km/s]'])
    int_t.append(t)
    int_rx.append(r.x)
    int_ry.append(r.y)
    int_rz.append(r.z)
    int_vx.append(v.x)
    int_vy.append(v.y)
    int_vz.append(v.z)
    int_erro_rx.append(100*abs((r.x-row['Rx [km]'])/row['Rx [km]']))
    int_erro_ry.append(100*abs((r.y-row['Ry [km]'])/row['Ry [km]']))
    int_erro_rz.append(100*abs((r.z-row['Rz [km]'])/row['Rz [km]']))
    int_erro_vx.append(100*abs((v.x-row['Vx [km/s]'])/row['Vx [km/s]']))
    int_erro_vy.append(100*abs((v.y-row['Vy [km/s]'])/row['Vy [km/s]']))
    int_erro_vz.append(100*abs((v.z-row['Vz [km/s]'])/row['Vz [km/s]']))

df_int = pd.DataFrame(
        data={
            "Time [s]": int_t,
            "Rx [km]": int_rx,
            "Ry [km]": int_ry,
            "Rz [km]": int_rz,
            "Vx [km/s]": int_vx,
            "Vy [km/s]": int_vy,
            "Vz [km/s]": int_vz,
        }
    )
df_erro = pd.DataFrame(
        data={
            "Time [s]": int_t,
            "Erro Rx [%]": int_erro_rx,
            "Erro Ry [%]": int_erro_ry,
            "Erro Rz [%]": int_erro_rz,
            "Erro Vx [%]": int_erro_vx,
            "Erro Vy [%]": int_erro_vy,
            "Erro Vz [%]": int_erro_vz,
        }
    )

In [12]:
df_erro.describe()

,Time [s],Erro Rx [%],Erro Ry [%],Erro Rz [%],Erro Vx [%],Erro Vy [%],Erro Vz [%]
count,8677.000000,8677.000000,8677.000000,8677.000000,8677.000000,8677.000000,8677.000000
mean,259241.178887,0.001363,0.001513,0.001905,0.040832,0.069385,0.086133
std,149668.027897,0.014853,0.026505,0.066015,0.266350,1.411592,0.852271
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,129673.527366,0.000115,0.000101,0.000119,0.003562,0.003707,0.003055
50%,259268.823305,0.000436,0.000423,0.000427,0.010400,0.010218,0.008421
75%,388814.526614,0.001155,0.001144,0.001091,0.022041,0.020995,0.020209
max,518418.130085,1.154494,2.187632,6.109203,12.840413,99.741661,46.905994


In [13]:
df_erro[df_erro['Erro Vy [%]'] == df_erro['Erro Vy [%]'].max()]

,Time [s],Erro Rx [%],Erro Ry [%],Erro Rz [%],Erro Vx [%],Erro Vy [%],Erro Vz [%]
5292,316234.135456,0.000446,0.000418,0.000005,0.040118,99.741661,0.002881


In [14]:
df_int[df_int['Time [s]'] == df_erro[df_erro['Erro Vy [%]'] == df_erro['Erro Vy [%]'].max()]['Time [s]'].values[0]]

,Time [s],Rx [km],Ry [km],Rz [km],Vx [km/s],Vy [km/s],Vz [km/s]
5292,316234.135456,4622.8511,-4962.561689,854.918086,-1.390205,0.000002,7.506973


In [17]:
# determining the name of the file
file_name = 'Dados\\Ephemeris\\Interpolation_Ephemeris.xlsx'
 
# saving the excel
df_int.to_excel(file_name)

In [16]:
plot_data(
    df_list=[df_60, df_int],
    legend=["Dados Originais do GMAT", "Dados Interpolados"],
    y_data=["Vz [km/s]","Vy [km/s]"],
    x_data='Time [s]',
    h_dist=False
)